# ✂️ RAG Chunking: Sztuka krojenia tekstu

Kiedy budujesz system RAG, nie wrzucasz całej książki do bazy wektorowej. Dzielisz ją na fragmenty.

**Dlaczego to trudne?**
Wyobraź sobie zdanie: *"Kluczem do sejfu jest... [CIĘCIE] ...1234."*
*   Chunk 1: "Kluczem do sejfu jest..." (AI nie wie co).
*   Chunk 2: "...1234." (AI nie wie, do czego to kod).

Oba kawałki stają się bezużyteczne.

**Strategie:**
1.  **Fixed Size:** Tniemy równo co 100 znaków (Ryzykowne).
2.  **Sliding Window (Overlap):** Tniemy co 100 znaków, ale cofamy się o 20. (Tworzymy zakładkę).
3.  **Recursive Split:** Najpierw tniemy akapity. Jak za duże -> tniemy zdania. Jak za duże -> tniemy słowa. (Standard w LangChain).

In [1]:
# 1. PRZYGOTOWANIE DANYCH
# Długi tekst (symulacja artykułu)

long_text = """
ROZDZIAŁ 1: WSTĘP DO AI
Sztuczna inteligencja zmienia świat w tempie wykładniczym. Modele językowe, takie jak GPT-4, potrafią pisać kod, wiersze i analizować prawo.
Jednakże, mają one swoje ograniczenia, zwane halucynacjami.

ROZDZIAŁ 2: PROBLEM PAMIĘCI
Modele nie mają pamięci długotrwałej. Kiedy zamykasz okno czatu, model zapomina, o czym rozmawialiście.
Dlatego stosujemy techniki takie jak RAG (Retrieval Augmented Generation), aby dostarczyć im kontekst.

ROZDZIAŁ 3: PRZYSZŁOŚĆ
W przyszłości agenci AI będą działać autonomicznie. Będą rezerwować loty i robić zakupy.
Ważne jest jednak bezpieczeństwo i etyka tych systemów.
"""

# Usuwamy zbędne puste linie dla czytelności
long_text = long_text.strip()
print(f"Długość tekstu: {len(long_text)} znaków.")

Długość tekstu: 629 znaków.


## Metoda 1: Naive Chunking (Sztywne cięcie)

Tniemy tekst równo co `chunk_size` znaków.
To najprostsza metoda, ale niszczy sens, jeśli cięcie wypadnie w środku ważnego słowa lub zdania.

In [2]:
def naive_chunking(text, chunk_size=100):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunk = text[i : i + chunk_size]
        chunks.append(chunk)
    return chunks

# Testujemy
chunks_naive = naive_chunking(long_text, chunk_size=120)

print("--- METODA NAIWNA ---")
for i, chunk in enumerate(chunks_naive):
    print(f"CHUNK {i}: [{chunk}]")

print("\nZAUWAŻ PROBLEM:")
print("Chunk 0 kończy się w połowie zdania. Chunk 1 zaczyna się od środka.")
print("Jeśli zapytasz AI o 'halucynacje' (koniec Chunk 0), może nie zrozumieć kontekstu z początku Chunk 1.")

--- METODA NAIWNA ---
CHUNK 0: [ROZDZIAŁ 1: WSTĘP DO AI
Sztuczna inteligencja zmienia świat w tempie wykładniczym. Modele językowe, takie jak GPT-4, pot]
CHUNK 1: [rafią pisać kod, wiersze i analizować prawo.
Jednakże, mają one swoje ograniczenia, zwane halucynacjami.

ROZDZIAŁ 2: PR]
CHUNK 2: [OBLEM PAMIĘCI
Modele nie mają pamięci długotrwałej. Kiedy zamykasz okno czatu, model zapomina, o czym rozmawialiście.
Dl]
CHUNK 3: [atego stosujemy techniki takie jak RAG (Retrieval Augmented Generation), aby dostarczyć im kontekst.

ROZDZIAŁ 3: PRZYSZ]
CHUNK 4: [ŁOŚĆ
W przyszłości agenci AI będą działać autonomicznie. Będą rezerwować loty i robić zakupy.
Ważne jest jednak bezpiecz]
CHUNK 5: [eństwo i etyka tych systemów.]

ZAUWAŻ PROBLEM:
Chunk 0 kończy się w połowie zdania. Chunk 1 zaczyna się od środka.
Jeśli zapytasz AI o 'halucynacje' (koniec Chunk 0), może nie zrozumieć kontekstu z początku Chunk 1.


## Metoda 2: Overlap (Zakładka)

Naprawiamy problem.
Tniemy kawałek o długości 120, ale następny zaczynamy nie od 121, tylko cofamy się o 20 znaków (Overlap).
Dzięki temu informacje z granicy cięcia pojawiają się w **obu** kawałkach.

In [3]:
def overlapping_chunking(text, chunk_size=120, overlap=30):
    chunks = []
    step = chunk_size - overlap  # O tyle się przesuwamy
    
    for i in range(0, len(text), step):
        chunk = text[i : i + chunk_size]
        chunks.append(chunk)
        
        # Jeśli doszliśmy do końca, przerywamy
        if i + chunk_size >= len(text):
            break
            
    return chunks

chunks_overlap = overlapping_chunking(long_text, chunk_size=120, overlap=30)

print("--- METODA Z ZAKŁADKĄ (OVERLAP) ---")
for i, chunk in enumerate(chunks_overlap):
    print(f"CHUNK {i}: [{chunk}]")
    
print("\nZALETA:")
print("Spójrz na koniec Chunk 0 i początek Chunk 1.")
print("Zdanie o halucynacjach jest powtórzone. Dzięki temu nie tracimy wątku!")

--- METODA Z ZAKŁADKĄ (OVERLAP) ---
CHUNK 0: [ROZDZIAŁ 1: WSTĘP DO AI
Sztuczna inteligencja zmienia świat w tempie wykładniczym. Modele językowe, takie jak GPT-4, pot]
CHUNK 1: [językowe, takie jak GPT-4, potrafią pisać kod, wiersze i analizować prawo.
Jednakże, mają one swoje ograniczenia, zwane ]
CHUNK 2: [one swoje ograniczenia, zwane halucynacjami.

ROZDZIAŁ 2: PROBLEM PAMIĘCI
Modele nie mają pamięci długotrwałej. Kiedy za]
CHUNK 3: [pamięci długotrwałej. Kiedy zamykasz okno czatu, model zapomina, o czym rozmawialiście.
Dlatego stosujemy techniki takie]
CHUNK 4: [atego stosujemy techniki takie jak RAG (Retrieval Augmented Generation), aby dostarczyć im kontekst.

ROZDZIAŁ 3: PRZYSZ]
CHUNK 5: [ kontekst.

ROZDZIAŁ 3: PRZYSZŁOŚĆ
W przyszłości agenci AI będą działać autonomicznie. Będą rezerwować loty i robić zaku]
CHUNK 6: [ą rezerwować loty i robić zakupy.
Ważne jest jednak bezpieczeństwo i etyka tych systemów.]

ZALETA:
Spójrz na koniec Chunk 0 i początek Chunk 1.
Zdanie o halucyna

## Metoda 3: Recursive Splitting (Metoda Profesjonalna)

Tak działają biblioteki typu **LangChain** czy **LlamaIndex**.
Algorytm nie tnie na ślepo po znakach.
Próbuje ciąć w "logicznych" miejscach:
1.  Najpierw szuka podwójnych enterów `\n\n` (Akapity).
2.  Jak akapit jest wciąż za duży, szuka pojedynczych enterów `\n` (Linie).
3.  Jak linia jest za duża, szuka spacji ` ` (Słowa).

In [4]:
def recursive_split(text, max_chunk_size=120):
    chunks = []
    
    # KROK 1: Podziel na akapity (Logiczne całości)
    paragraphs = text.split('\n\n')
    
    current_chunk = ""
    
    for para in paragraphs:
        # Sprawdzamy, czy jak dodamy ten akapit, to przekroczymy limit?
        if len(current_chunk) + len(para) <= max_chunk_size:
            # Mieści się -> Doklejamy
            current_chunk += para + "\n\n"
        else:
            # Nie mieści się -> Zapisujemy stary chunk i zaczynamy nowy
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = para + "\n\n"
            
            # (Tutaj w prawdziwym LangChain nastąpiłoby dalsze cięcie akapitu na zdania,
            #  jeśli sam akapit byłby większy niż max_chunk_size. Dla uproszczenia to pomijamy).
    
    # Dodajemy ostatni kawałek
    if current_chunk:
        chunks.append(current_chunk.strip())
        
    return chunks

chunks_recursive = recursive_split(long_text, max_chunk_size=150)

print("--- METODA REKURENCYJNA (Logiczna) ---")
for i, chunk in enumerate(chunks_recursive):
    print(f"CHUNK {i}:\n{chunk}")
    print("-" * 20)

print("WYNIK:")
print("Każdy chunk to pełny, logiczny rozdział. Nie ucięliśmy żadnego zdania w połowie!")

--- METODA REKURENCYJNA (Logiczna) ---
CHUNK 0:
ROZDZIAŁ 1: WSTĘP DO AI
Sztuczna inteligencja zmienia świat w tempie wykładniczym. Modele językowe, takie jak GPT-4, potrafią pisać kod, wiersze i analizować prawo.
Jednakże, mają one swoje ograniczenia, zwane halucynacjami.
--------------------
CHUNK 1:
ROZDZIAŁ 2: PROBLEM PAMIĘCI
Modele nie mają pamięci długotrwałej. Kiedy zamykasz okno czatu, model zapomina, o czym rozmawialiście.
Dlatego stosujemy techniki takie jak RAG (Retrieval Augmented Generation), aby dostarczyć im kontekst.
--------------------
CHUNK 2:
ROZDZIAŁ 3: PRZYSZŁOŚĆ
W przyszłości agenci AI będą działać autonomicznie. Będą rezerwować loty i robić zakupy.
Ważne jest jednak bezpieczeństwo i etyka tych systemów.
--------------------
WYNIK:
Każdy chunk to pełny, logiczny rozdział. Nie ucięliśmy żadnego zdania w połowie!


## 🧠 Podsumowanie: Jak nie zepsuć RAG-a?

Wybór strategii zależy od danych:

1.  **Kod źródłowy (Python, JS):** Tniemy w oparciu o klasy i funkcje. (Overlap mało ważny).
2.  **Dokumenty prawne/umowy:** Bardzo duży **Overlap** (nawet 50%), bo kontekst jednego paragrafu może zależeć od poprzedniego zdania.
3.  **Wiki/Artykuły:** Recursive Split (zachowanie akapitów) jest najlepszy.

**Tu jest haczyk (Embedding Model).**
Pamiętaj, że każdy model Embeddings ma limit (np. 512 lub 8192 tokenów). Twój Chunk **musi** być mniejszy niż ten limit. Inaczej model utnie końcówkę i nawet o tym nie będziesz wiedział.